<a href="https://colab.research.google.com/github/geopan2000/Loan-Approval-Prediction/blob/main/LoanApprovalPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [305]:
# Clone the GitHub repository
!git clone https://github.com/geopan2000/Loan-Approval-Prediction.git

# Change directory to the cloned repository
%cd Loan-Approval-Prediction

Cloning into 'Loan-Approval-Prediction'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 55 (delta 25), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (55/55), 2.12 MiB | 5.49 MiB/s, done.
Resolving deltas: 100% (25/25), done.
/content/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction/Loan-Approval-Prediction


In [306]:
import pandas as pd
import numpy as np
!pip install xgboost
import xgboost as xgb

!pip install imbalanced-learn
!pip install scikit-optimize
!pip install shap
from skopt import BayesSearchCV
from skopt.space import Real, Integer

from sklearn.model_selection import KFold
from skopt import BayesSearchCV
from skopt.space import Real, Integer
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from google.colab import files
from sklearn.metrics import roc_auc_score, classification_report

In [307]:
train_set = pd.read_csv('/content/Loan-Approval-Prediction/data/train.csv') # Load train set
test_set = pd.read_csv('/content/Loan-Approval-Prediction/data/test.csv') #Load test set

In [308]:
print(train_set.isna().sum()) #inspect train set for possible missing values
print(test_set.isna().sum()) #inspect test set for possible missing values

id                            0
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
loan_status                   0
dtype: int64
id                            0
person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
dtype: int64


In [309]:
print(train_set.dtypes) #inspect data types

id                              int64
person_age                      int64
person_income                   int64
person_home_ownership          object
person_emp_length             float64
loan_intent                    object
loan_grade                     object
loan_amnt                       int64
loan_int_rate                 float64
loan_percent_income           float64
cb_person_default_on_file      object
cb_person_cred_hist_length      int64
loan_status                     int64
dtype: object


In [310]:
print('The size of the train set is:', train_set.shape)
print('The size of the test set is:', test_set.shape) # the target variable is missing from the set

The size of the train set is: (58645, 13)
The size of the test set is: (39098, 12)


**Inspect which features are object type and what are the differnet values inside them**

In [311]:
print(train_set.info())
print(train_set['person_home_ownership'].value_counts())
print(train_set['loan_intent'].value_counts())
print(train_set['loan_grade'].value_counts())
print(train_set['cb_person_default_on_file'].value_counts())
print(train_set['loan_status'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58645 entries, 0 to 58644
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          58645 non-null  int64  
 1   person_age                  58645 non-null  int64  
 2   person_income               58645 non-null  int64  
 3   person_home_ownership       58645 non-null  object 
 4   person_emp_length           58645 non-null  float64
 5   loan_intent                 58645 non-null  object 
 6   loan_grade                  58645 non-null  object 
 7   loan_amnt                   58645 non-null  int64  
 8   loan_int_rate               58645 non-null  float64
 9   loan_percent_income         58645 non-null  float64
 10  cb_person_default_on_file   58645 non-null  object 
 11  cb_person_cred_hist_length  58645 non-null  int64  
 12  loan_status                 58645 non-null  int64  
dtypes: float64(3), int64(6), object

**Feature Engineering**

In [312]:
# Ensure to map and assign the results back to the columns
train_set['cb_person_default_on_file'] = train_set['cb_person_default_on_file'].map({'Y': 1, 'N': 0}).astype(np.float32)

# Keep a copy of the original loan_grade column before encoding
train_set['loan_grade_0'] = train_set['loan_grade']

train_set['loan_grade'] = train_set['loan_grade'].map({'A': 0.1, 'B': 0.2, 'C': 0.4, 'D': 0.5, 'E': 0.6, 'F': 0.8, 'G': 1}).astype(np.float32)

# Dummy encoding for categorical columns
train_set = pd.get_dummies(train_set, columns=['loan_grade_0', 'person_home_ownership', 'loan_intent'])

# Drop the 'id' column as it's not needed
train_set.drop(['id'], axis=1, inplace=True)

# Verify data types after transformations
print(train_set['loan_grade'].head())
print(train_set['cb_person_default_on_file'].head())
print(train_set.info())  # To check if all object columns have been transformed

# Now apply the same steps to the test set
test_id = test_set['id']  # Store test IDs for final submission

test_set['cb_person_default_on_file'] = test_set['cb_person_default_on_file'].map({'Y': 1, 'N': 0}).astype(np.float32)

test_set['loan_grade_0'] = test_set['loan_grade']

test_set['loan_grade'] = test_set['loan_grade'].map({'A': 0.1, 'B': 0.2, 'C': 0.4, 'D': 0.5, 'E': 0.6, 'F': 0.8, 'G': 1}).astype(np.float32)

test_set = pd.get_dummies(test_set, columns=['loan_grade_0', 'person_home_ownership', 'loan_intent'])
test_set.drop(['id'], axis=1, inplace=True)

# Verify data types after transformations for the test set
print(test_set.info())


0    0.2
1    0.4
2    0.1
3    0.2
4    0.1
Name: loan_grade, dtype: float32
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: cb_person_default_on_file, dtype: float32
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58645 entries, 0 to 58644
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   person_age                      58645 non-null  int64  
 1   person_income                   58645 non-null  int64  
 2   person_emp_length               58645 non-null  float64
 3   loan_grade                      58645 non-null  float32
 4   loan_amnt                       58645 non-null  int64  
 5   loan_int_rate                   58645 non-null  float64
 6   loan_percent_income             58645 non-null  float64
 7   cb_person_default_on_file       58645 non-null  float32
 8   cb_person_cred_hist_length      58645 non-null  int64  
 9   loan_status                     58645 non-nu

**Feature Engineering: Part2-create new features that might help increasing the accuracy**

In [313]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import pandas as pd

# Separate features and target
y = train_set['loan_status']
X = train_set.drop(['loan_status'], axis=1)

# Initial split for training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

# Initialize SMOTE to oversample the minority class
smote = SMOTE(sampling_strategy=0.25, random_state=42)

# Oversample the training set
X_train, y_train = smote.fit_resample(X_train, y_train)

# Convert back to DataFrame to maintain column names
X_train = pd.DataFrame(X_train, columns=X.columns)
X_val = pd.DataFrame(X_val, columns=X.columns)

# Shuffle the training and validation sets after resampling
X_train, y_train = X_train.sample(frac=1, random_state=42).reset_index(drop=True), y_train.sample(frac=1, random_state=42).reset_index(drop=True)
X_val, y_val = X_val.sample(frac=1, random_state=42).reset_index(drop=True), y_val.sample(frac=1, random_state=42).reset_index(drop=True)

X_test = test_set.copy()

# Print the count of True and False in both train and validation sets
print("Training set class distribution:\n", y_train.value_counts())
print("Validation set class distribution:\n", y_val.value_counts())


Training set class distribution:
 loan_status
0    45265
1    11316
Name: count, dtype: int64
Validation set class distribution:
 loan_status
0    5030
1     835
Name: count, dtype: int64


 **Predicting Using XGB**

In [314]:
# Define the model with the provided parameters
xgb_model1 = xgb.XGBClassifier(
    colsample_bytree=0.607686819159888,
    gamma=0.0,
    learning_rate=0.1386554508211204,
    max_depth=5,
    n_estimators=444,
    reg_alpha=0.0,
    reg_lambda=10.0,
    subsample=1.0,
    eval_metric='auc'
)

xgb_model1.fit(X_train, y_train)

y_pred_prob_XGB1_train = xgb_model1.predict_proba(X_train)[:, 1]
print(roc_auc_score(y_train, y_pred_prob_XGB1_train))

y_pred_prob_XGB1_val = xgb_model1.predict_proba(X_val)[:, 1]
print(roc_auc_score(y_val, y_pred_prob_XGB1_val))

y_pred_prob_XGB1_test = xgb_model1.predict_proba(X_test)[:, 1]

0.9863323909234558
0.9592792942941155


In [315]:
# Define the model with the provided parameters
xgb_model2 = xgb.XGBClassifier(
    colsample_bytree=0.5349230389326496,
    gamma=0.0,
    learning_rate=0.12008507715256864,
    max_depth=5,
    n_estimators=382,
    reg_alpha=0.0,
    reg_lambda=4.1310919699589626,
    subsample=1.0,
    eval_metric='auc'
)

xgb_model2.fit(X_train, y_train)

y_pred_prob_XGB2_train = xgb_model2.predict_proba(X_train)[:, 1]
print(roc_auc_score(y_train, y_pred_prob_XGB2_train))

y_pred_prob_XGB2_val = xgb_model2.predict_proba(X_val)[:, 1]
print(roc_auc_score(y_val, y_pred_prob_XGB2_val))

y_pred_prob_XGB2_test = xgb_model2.predict_proba(X_test)[:, 1]

0.9853568418836062
0.9595164343281629


In [316]:
# Define the model with the provided parameters
xgb_model3 = xgb.XGBClassifier(
    colsample_bytree=0.5,
    gamma=0.0,
    learning_rate=0.2590922138514278,
    max_depth=3,
    n_estimators=432,
    reg_alpha=0.0,
    reg_lambda=6.510926588807951,
    subsample=1.0,
    eval_metric='auc'
)

xgb_model3.fit(X_train, y_train)

y_pred_prob_XGB3_train = xgb_model3.predict_proba(X_train)[:, 1]
print(roc_auc_score(y_train, y_pred_prob_XGB3_train))

y_pred_prob_XGB3_val = xgb_model3.predict_proba(X_val)[:, 1]
print(roc_auc_score(y_val, y_pred_prob_XGB3_val))

y_pred_prob_XGB3_test = xgb_model3.predict_proba(X_test)[:, 1]

0.9802396130606232
0.959690956060047


In [317]:
from sklearn.metrics import roc_auc_score

# Define the model with the 4th model's parameters
xgb_model4 = xgb.XGBClassifier(
    colsample_bytree=0.5,
    gamma=0.0,
    learning_rate=0.14243206021024543,
    max_depth=5,
    n_estimators=398,
    reg_alpha=0.0,
    reg_lambda=10.0,
    subsample=0.7923461422010261,
    eval_metric='auc'
)

# Train the model on the training data
xgb_model4.fit(X_train, y_train)

# Predict probabilities on training set and calculate AUC
y_pred_prob_XGB4_train = xgb_model4.predict_proba(X_train)[:, 1]
print("Training AUC:", roc_auc_score(y_train, y_pred_prob_XGB4_train))

# Predict probabilities on validation set and calculate AUC
y_pred_prob_XGB4_val = xgb_model4.predict_proba(X_val)[:, 1]
print("Validation AUC:", roc_auc_score(y_val, y_pred_prob_XGB4_val))

# Predict probabilities on test set
y_pred_prob_XGB4_test = xgb_model4.predict_proba(X_test)[:, 1]


Training AUC: 0.9859583768840633
Validation AUC: 0.9586043023297343


In [318]:
from sklearn.metrics import roc_auc_score

# Define the model with the 5th model's parameters
xgb_model5 = xgb.XGBClassifier(
    colsample_bytree=0.5,
    gamma=0.0,
    learning_rate=0.15072085781971037,
    max_depth=4,
    n_estimators=440,
    reg_alpha=0.0,
    reg_lambda=10.0,
    subsample=0.9001669533542271,
    eval_metric='auc'
)

# Train the model on the training data
xgb_model5.fit(X_train, y_train)

# Predict probabilities on the training set and calculate AUC
y_pred_prob_XGB5_train = xgb_model5.predict_proba(X_train)[:, 1]
print("Training AUC:", roc_auc_score(y_train, y_pred_prob_XGB5_train))

# Predict probabilities on the validation set and calculate AUC
y_pred_prob_XGB5_val = xgb_model5.predict_proba(X_val)[:, 1]
print("Validation AUC:", roc_auc_score(y_val, y_pred_prob_XGB5_val))

# Predict probabilities on the test set
y_pred_prob_XGB5_test = xgb_model5.predict_proba(X_test)[:, 1]


Training AUC: 0.9821247715380348
Validation AUC: 0.958890965583743


In [319]:
from sklearn.metrics import roc_auc_score

# Define the model with the 6th model's parameters
xgb_model6 = xgb.XGBClassifier(
    colsample_bytree=0.5,
    gamma=0.0,
    learning_rate=0.1229119989783978,
    max_depth=5,
    n_estimators=364,
    reg_alpha=0.0,
    reg_lambda=10.0,
    subsample=1.0,
    eval_metric='auc'
)

# Train the model on the training data
xgb_model6.fit(X_train, y_train)

# Predict probabilities on the training set and calculate AUC
y_pred_prob_XGB6_train = xgb_model6.predict_proba(X_train)[:, 1]
print("Training AUC:", roc_auc_score(y_train, y_pred_prob_XGB6_train))

# Predict probabilities on the validation set and calculate AUC
y_pred_prob_XGB6_val = xgb_model6.predict_proba(X_val)[:, 1]
print("Validation AUC:", roc_auc_score(y_val, y_pred_prob_XGB6_val))

# Predict probabilities on the test set
y_pred_prob_XGB6_test = xgb_model6.predict_proba(X_test)[:, 1]


Training AUC: 0.9829932159452033
Validation AUC: 0.9597547648242284


In [320]:
from sklearn.metrics import roc_auc_score

# Define the model with the 7th model's parameters
xgb_model7 = xgb.XGBClassifier(
    colsample_bytree=1.0,
    gamma=0.0,
    learning_rate=0.1280840580147865,
    max_depth=4,
    n_estimators=392,
    reg_alpha=0.0,
    reg_lambda=5.661309772706808,
    subsample=0.8817289754308963,
    eval_metric='auc'
)

# Train the model on the training data
xgb_model7.fit(X_train, y_train)

# Predict probabilities on the training set and calculate AUC
y_pred_prob_XGB7_train = xgb_model7.predict_proba(X_train)[:, 1]
print("Training AUC:", roc_auc_score(y_train, y_pred_prob_XGB7_train))

# Predict probabilities on the validation set and calculate AUC
y_pred_prob_XGB7_val = xgb_model7.predict_proba(X_val)[:, 1]
print("Validation AUC:", roc_auc_score(y_val, y_pred_prob_XGB7_val))

# Predict probabilities on the test set
y_pred_prob_XGB7_test = xgb_model7.predict_proba(X_test)[:, 1]


Training AUC: 0.9825542384489875
Validation AUC: 0.9588381090701301


In [321]:
from sklearn.metrics import roc_auc_score

# Define the model with the 8th model's parameters
xgb_model8 = xgb.XGBClassifier(
    colsample_bytree=1.0,
    gamma=0.0,
    learning_rate=0.17242897272288477,
    max_depth=4,
    n_estimators=500,
    reg_alpha=0.0,
    reg_lambda=10.0,
    subsample=1.0,
    eval_metric='auc'
)

# Train the model on the training data
xgb_model8.fit(X_train, y_train)

# Predict probabilities on the training set and calculate AUC
y_pred_prob_XGB8_train = xgb_model8.predict_proba(X_train)[:, 1]
print("Training AUC:", roc_auc_score(y_train, y_pred_prob_XGB8_train))

# Predict probabilities on the validation set and calculate AUC
y_pred_prob_XGB8_val = xgb_model8.predict_proba(X_val)[:, 1]
print("Validation AUC:", roc_auc_score(y_val, y_pred_prob_XGB8_val))

# Predict probabilities on the test set
y_pred_prob_XGB8_test = xgb_model8.predict_proba(X_test)[:, 1]


Training AUC: 0.9850852147658636
Validation AUC: 0.959015011726051


In [322]:
from sklearn.metrics import roc_auc_score

# Define the model with the 9th model's parameters
xgb_model9 = xgb.XGBClassifier(
    colsample_bytree=0.8466482373859222,
    gamma=0.444689170727064,
    learning_rate=0.25750013975195113,
    max_depth=3,
    n_estimators=480,
    reg_alpha=0.3133214121333228,
    reg_lambda=9.738406839866839,
    subsample=0.9701237328878916,
    eval_metric='auc'
)

# Train the model on the training data
xgb_model9.fit(X_train, y_train)

# Predict probabilities on the training set and calculate AUC
y_pred_prob_XGB9_train = xgb_model9.predict_proba(X_train)[:, 1]
print("Training AUC:", roc_auc_score(y_train, y_pred_prob_XGB9_train))

# Predict probabilities on the validation set and calculate AUC
y_pred_prob_XGB9_val = xgb_model9.predict_proba(X_val)[:, 1]
print("Validation AUC:", roc_auc_score(y_val, y_pred_prob_XGB9_val))

# Predict probabilities on the test set
y_pred_prob_XGB9_test = xgb_model9.predict_proba(X_test)[:, 1]


Training AUC: 0.9810140097568473
Validation AUC: 0.9593259604052332


In [323]:
from sklearn.metrics import roc_auc_score

# Define the model with the 10th model's parameters
xgb_model10 = xgb.XGBClassifier(
    colsample_bytree=0.5,
    gamma=0.0,
    learning_rate=0.1353854406983619,
    max_depth=4,
    n_estimators=477,
    reg_alpha=0.017833537744749474,
    reg_lambda=0.0,
    subsample=1.0,
    eval_metric='auc'
)

# Train the model on the training data
xgb_model10.fit(X_train, y_train)

# Predict probabilities on the training set and calculate AUC
y_pred_prob_XGB10_train = xgb_model10.predict_proba(X_train)[:, 1]
print("Training AUC:", roc_auc_score(y_train, y_pred_prob_XGB10_train))

# Predict probabilities on the validation set and calculate AUC
y_pred_prob_XGB10_val = xgb_model10.predict_proba(X_val)[:, 1]
print("Validation AUC:", roc_auc_score(y_val, y_pred_prob_XGB10_val))

# Predict probabilities on the test set
y_pred_prob_XGB10_test = xgb_model10.predict_proba(X_test)[:, 1]


Training AUC: 0.9852708005177632
Validation AUC: 0.9611235580528803


**Adding the predictions as new features**

In [324]:
X_train['y_pred_prob_XGB1'] = y_pred_prob_XGB1_train
X_val['y_pred_prob_XGB1'] = y_pred_prob_XGB1_val
X_test['y_pred_prob_XGB1'] = y_pred_prob_XGB1_test

X_train['y_pred_prob_XGB2'] = y_pred_prob_XGB2_train
X_val['y_pred_prob_XGB2'] = y_pred_prob_XGB2_val
X_test['y_pred_prob_XGB2'] = y_pred_prob_XGB2_test

X_train['y_pred_prob_XGB3'] = y_pred_prob_XGB3_train
X_val['y_pred_prob_XGB3'] = y_pred_prob_XGB3_val
X_test['y_pred_prob_XGB3'] = y_pred_prob_XGB3_test

X_train['y_pred_prob_XGB4'] = y_pred_prob_XGB4_train
X_val['y_pred_prob_XGB4'] = y_pred_prob_XGB4_val
X_test['y_pred_prob_XGB4'] = y_pred_prob_XGB4_test

X_train['y_pred_prob_XGB5'] = y_pred_prob_XGB5_train
X_val['y_pred_prob_XGB5'] = y_pred_prob_XGB5_val
X_test['y_pred_prob_XGB5'] = y_pred_prob_XGB5_test

X_train['y_pred_prob_XGB6'] = y_pred_prob_XGB6_train
X_val['y_pred_prob_XGB6'] = y_pred_prob_XGB6_val
X_test['y_pred_prob_XGB6'] = y_pred_prob_XGB6_test

X_train['y_pred_prob_XGB7'] = y_pred_prob_XGB7_train
X_val['y_pred_prob_XGB7'] = y_pred_prob_XGB7_val
X_test['y_pred_prob_XGB7'] = y_pred_prob_XGB7_test

X_train['y_pred_prob_XGB8'] = y_pred_prob_XGB8_train
X_val['y_pred_prob_XGB8'] = y_pred_prob_XGB8_val
X_test['y_pred_prob_XGB8'] = y_pred_prob_XGB8_test

X_train['y_pred_prob_XGB9'] = y_pred_prob_XGB9_train
X_val['y_pred_prob_XGB9'] = y_pred_prob_XGB9_val
X_test['y_pred_prob_XGB9'] = y_pred_prob_XGB9_test

X_train['y_pred_prob_XGB10'] = y_pred_prob_XGB10_train
X_val['y_pred_prob_XGB10'] = y_pred_prob_XGB10_val
X_test['y_pred_prob_XGB10'] = y_pred_prob_XGB10_test

In [325]:
y_pred_prob_XGB_train = (y_pred_prob_XGB1_train + y_pred_prob_XGB2_train + y_pred_prob_XGB3_train +
                         y_pred_prob_XGB4_train + y_pred_prob_XGB5_train + y_pred_prob_XGB6_train +
                         y_pred_prob_XGB7_train + y_pred_prob_XGB8_train + y_pred_prob_XGB8_train +
                         y_pred_prob_XGB9_train + y_pred_prob_XGB10_train) / 10

print("Validation AUC:", roc_auc_score(y_train, y_pred_prob_XGB_train))

y_pred_prob_XGB_val = (y_pred_prob_XGB1_val + y_pred_prob_XGB2_val + y_pred_prob_XGB3_val +
                       y_pred_prob_XGB4_val + y_pred_prob_XGB5_val + y_pred_prob_XGB6_val +
                       y_pred_prob_XGB7_val + y_pred_prob_XGB8_val + y_pred_prob_XGB8_val +
                       y_pred_prob_XGB9_val + y_pred_prob_XGB10_val) / 10

print("Validation AUC:", roc_auc_score(y_val, y_pred_prob_XGB_val))

y_pred_prob_XGB_test = (y_pred_prob_XGB1_test + y_pred_prob_XGB2_test + y_pred_prob_XGB3_test +
                       y_pred_prob_XGB4_test + y_pred_prob_XGB5_test + y_pred_prob_XGB6_test +
                       y_pred_prob_XGB7_test + y_pred_prob_XGB8_test + y_pred_prob_XGB8_test +
                       y_pred_prob_XGB9_test + y_pred_prob_XGB10_test) / 10

Validation AUC: 0.9846403462317681
Validation AUC: 0.9600292853656504


In [326]:
# Prepare the output DataFrame
output = pd.DataFrame({'id': test_id.values, 'loan_status': y_pred_prob_XGB_test})

# Remove any duplicate rows by 'PassengerId'
output.drop_duplicates(subset='id', keep='first', inplace=True)

# Save predictions to a CSV file
output.to_csv('predictions.csv', index=False)
files.download('predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [257]:
from sklearn.model_selection import KFold
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import xgboost as xgb
import pandas as pd

# Define parameter search space
search_spaces = {
    'learning_rate': Real(0.001, 0.5, 'uniform'),
    'max_depth': Integer(2, 7),
    'n_estimators': Integer(100, 500),
    'subsample': Real(0.5, 1.0, 'uniform'),
    'colsample_bytree': Real(0.5, 1.0, 'uniform'),
    'gamma': Real(0, 10, 'uniform'),
    'reg_alpha': Real(0, 10, 'uniform'),
    'reg_lambda': Real(0, 10, 'uniform')
}

# Initialize the XGBClassifier model with GPU support
model = xgb.XGBClassifier(
    objective='binary:logistic',  # Use for binary classification
    tree_method='hist',  # Use GPU for training
    verbosity=2
)

kf = KFold(n_splits=5)  # 5-fold cross-validation

# Using 'roc_auc' as the scoring metric
optimizer = BayesSearchCV(
    estimator=model,
    search_spaces=search_spaces,
    n_iter=124,
    cv=kf,
    scoring='roc_auc',
    verbose=1,
    n_jobs=-1,
    return_train_score=True
)

# Fit the optimizer using X_train and y_train
optimizer.fit(X_train, y_train)

# Get cross-validation results
cv_results = optimizer.cv_results_

# Create a DataFrame to sort and filter the best models
results_df = pd.DataFrame(cv_results)
top_5_results = results_df.nlargest(5, 'mean_test_score')  # Get top 10 models by mean test score

# Train each of the top 10 models on the full training data and save them
top_5_models = []
for i, row in top_5_results.iterrows():
    params = row['params']
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        tree_method='hist',
        **params
    )
    model.fit(X_train, y_train)
    top_5_models.append(model)

# Optionally, print parameters and scores for the top 10 models
for idx, model in enumerate(top_5_models):
    print(f"Model {idx + 1} parameters: {top_5_results.iloc[idx]['params']}")
    print(f"Model {idx + 1} CV score: {top_5_results.iloc[idx]['mean_test_score']}")

print("Top 10 XGBoost models have been trained and saved.")


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi